In [79]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

import sklearn
import scipy.sparse

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import seaborn as sns
from tqdm import tqdm_notebook

In [3]:
DATA_FOLDER = './readonly/'
RESULTS_FOLDER = './results/'

data = pd.read_excel(os.path.join(DATA_FOLDER, 'MFTRAVEL_HACKATHON.xlsx'), sheet_name=None)

In [61]:
countries = data['CountryDict']['CNTRY_CODE'].values.astype('int32')
data.keys()

odict_keys(['Data', 'CountryDict', 'FieldsDiscr'])

In [62]:
countries

array([        895,           4,           8,          12,          16,
                20,          24,         660,          10,          28,
                32,          51,         533,          36,          40,
                31,          44,          48,          50,          52,
               112,          56,          84,         204,          60,
                64,          68,         535,          70,          72,
                74,          76,          92,          96,         100,
               854,         108,         116,         120,         124,
               132,         136,         140,         148,         152,
               156,         170,         174,         178,         184,
               188,         384,         191,         192,         531,
               196,         203,         408,         626,         180,
               208,          86,         262,         212,         214,
               218,         818,         222,         226,      

In [113]:
MaxFlightPrice = 100000
MaxHotelPrice = 100000
MaxHotelPrice = 100000
MaxRandDays = 500
MaxHotels = 500

class Environment:
    def gen_dataset(self):
        k = dict()
        hotels = ['Baba', 'Bubu', 'Kata']
        hotel_ind = 0
        start_date = pd.to_datetime('8/2/2020')

        print('Generating country_to_hotels dataset...')
        
        for i in tqdm_notebook(countries):
            country_to_hotels = {'name' : [], 'day_price': [], 'available_from' : [], 'available_to' : []}

            for _ in range(MaxHotels):
                country_to_hotels['name'].append('{}_{}'.format(hotels[hotel_ind % 3], hotel_ind))
                country_to_hotels['day_price'].append(np.random.randint(MaxHotelPrice))

                delta1 = np.random.randint(MaxRandDays)
                delta2 = np.random.randint(MaxRandDays//2)
                country_to_hotels['available_from'].append(start_date + pd.DateOffset(days = delta1))
                country_to_hotels['available_to'].append(start_date + pd.DateOffset(days = delta1 + delta2))

                hotel_ind+=1

            k[i] = pd.DataFrame(data=country_to_hotels)
            
        self.country_to_hotels_ = k
        
    def __init__(self, country_to_hotels = None):
        print('Generating time_to_planes dataset...')
        
        d = {'time' : [], 'price': [], 'from': [], 'to' : []}
        start_date = pd.to_datetime('8/2/2020')
        
        for i in range(100000):
            if i % 10 == 0: 
                start_date += pd.DateOffset(days = 1)
                
            d['time'].append(start_date)
            d['price'].append(np.random.randint(MaxFlightPrice))
            d['from'].append(countries[np.random.randint(countries.size)])
            d['to'].append(countries[np.random.randint(countries.size)])

        self.planes_timetable_ = pd.DataFrame(data=d).set_index('time')
    
        if (country_to_hotels == None):
            self.gen_dataset()
        else:
            self.country_to_hotels_ = country_to_hotels
        
        
        print('Done')
        
    def get_tickets(self, time1, time2):
        return self.planes_timetable[time1 : time2]
    
    def get_hotels(self, country_id, time1, time2): #datetime!!!
        return self.country_to_hotels_[country_id][
            time1 < self.country_to_hotels_[country_id]['available_from'] and 
            time2 > self.country_to_hotels_[country_id]['available_from'] ] 

In [114]:
env = Environment()

Generating time_to_planes dataset...
Generating country_to_hotels dataset...



Done


In [ ]:
save = env.country_to_hotels_

In [115]:
time1 = pd.to_datetime('20/2/2020')
time2 = pd.to_datetime('27/2/2020')

env.get_hotels(4, time1, time2)
env.get_tickets(time1, time2)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [101]:
countries

array([        895,           4,           8,          12,          16,
                20,          24,         660,          10,          28,
                32,          51,         533,          36,          40,
                31,          44,          48,          50,          52,
               112,          56,          84,         204,          60,
                64,          68,         535,          70,          72,
                74,          76,          92,          96,         100,
               854,         108,         116,         120,         124,
               132,         136,         140,         148,         152,
               156,         170,         174,         178,         184,
               188,         384,         191,         192,         531,
               196,         203,         408,         626,         180,
               208,          86,         262,         212,         214,
               218,         818,         222,         226,      